In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# 1. Upload Excel file
file_path = "../dati/intevernti_cleaned.xlsx"  # Modifica con il percorso corretto
df = pd.read_excel(file_path)
print(df.shape)
# 2. # Identify diagnoses that appear only once for each DEPARTMENT and apply clustering separately
cluster_map = {}

for reparto, df_reparto in df.groupby("REPARTO"):
    counts = df_reparto["DESCDIAGNOSI1"].value_counts()
    unique_values = counts[counts == 1].index  # unique value
    
    # Create a subset of the dataframe with only these elements
    df_unique = df_reparto[df_reparto["DESCDIAGNOSI1"].isin(unique_values)].copy()
    
    if not df_unique.empty:
        # Convert text to numbers using TF-IDF
        vectorizer = TfidfVectorizer(stop_words="english")
        X = vectorizer.fit_transform(df_unique["DESCDIAGNOSI1"].astype(str))
        
        # Apply K-Means with maximum 3 clusters or less if there are fewer elements
        num_clusters = min(3, len(df_unique))
        if num_clusters > 1:
            kmeans = KMeans(n_clusters=num_clusters, random_state=42)
            df_unique["Cluster"] = kmeans.fit_predict(X)
            
            # Update the "DESCDIAGNOSI1" column in the original dataframe
            for idx, row in df_unique.iterrows():
                cluster_map[row["DESCDIAGNOSI1"]] = f"{reparto}_Cluster_{row['Cluster']}"

# 3. Replace original values ​​with cluster name only for unique items
df["DESCDIAGNOSI1"] = df["DESCDIAGNOSI1"].apply(lambda x: cluster_map.get(x, x))

# 4. Save the updated file
df.to_excel("clustered_interventions_by_department.xlsx", index=False)



(10836, 32)
File aggiornato con i cluster per gli elementi unici all'interno di ciascun REPARTO, senza mescolare reparti diversi.


## remove outlier

In [ ]:
# Convert dates
df['INGRESSOSALA'] = pd.to_datetime(df['INGRESSOSALA'], format='%d/%m/%Y %H:%M')
df['USCITASALA'] = pd.to_datetime(df['USCITASALA'], format='%d/%m/%Y %H:%M')

# compute duration in minutes
df['DURATA'] = (df['USCITASALA'] - df['INGRESSOSALA']).dt.total_seconds() / 60

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Using Seaborn to plot point distribution
plt.figure(figsize=(8, 6))
sns.histplot(df['DURATA'], kde=True, bins=10, color='skyblue')

plt.title('Distribution of Points')
plt.xlabel('Duration')
plt.ylabel('Items')

plt.show()

# Plot histogram of durations
plt.figure(figsize=(10, 6))
plt.hist(df['DURATA'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of the duration of the interventions', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.ylabel('Frequence', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Boxplot of durations
plt.figure(figsize=(8, 6))
plt.boxplot(df['DURATA'], vert=False, patch_artist=True, 
            boxprops=dict(facecolor='lightblue', color='blue'),
            whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
            medianprops=dict(color='red'))
plt.title('Boxplot of the duration of the interventions', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.show()


File aggiornato senza outlier salvato come 'interventi_senza_outlier.xlsx'.


In [ ]:


# Identifying outliers with the IQR method
Q1 = df['DURATA'].quantile(0.25)
Q3 = df['DURATA'].quantile(0.75)
IQR = Q3 - Q1

# Define limits to detect outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the data by removing outliers
df_filtered = df[(df['DURATA'] >= lower_bound) & (df['DURATA'] <= upper_bound)]

# Save the new file without outliers
df_filtered.to_excel("interventions_per_department_without_outliers.xlsx", index=False)
